<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/extract_data/Extract_NOAA_Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# Pandas allows us to create a spreadsheet output
# datasets
import pandas

# The colab module to access data from your google drive
from google.colab import drive

In [ ]:
try:
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools.convert_types

In [ ]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [ ]:
drive.mount("/content/drive")

In [ ]:
# The region of interest
vec_roi_file = "/content/drive/MyDrive/burnt_veg/burnt_area_roi.geojson"


# Dates before the fire
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2024, month=5, day=28)

# No Data Value
out_no_data_val = 0.0

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_roi_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
gee_bbox_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [ ]:
roi_ee_poly = pb_gee_tools.convert_types.convert_vector_to_gee_polygon(vec_file = vec_roi_file)

In [ ]:
temp_img_col = ee.ImageCollection('NOAA/CPC/Temperature').filterBounds(gee_bbox_aoi).filter(ee.Filter.date(start_date, end_date))

In [ ]:
def create_time_series(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=roi_ee_poly)
    return ee.Feature(None, {"date": image.date().format(), "tmax": stats.get("tmax"), "tmin": stats.get("tmin")})

In [ ]:
temp_time_series = temp_img_col.map(create_time_series).getInfo()

In [ ]:
# Convert time series to a pandas DataFrame
temp_feats = temp_time_series["features"]

# Create a list of the dates and temperature data
temp_data = list()
for temp_feat in temp_feats:
    if "date" in temp_feat["properties"]:
        temp_data.append(
            {
                "date": temp_feat["properties"]["date"],
                "tmax": temp_feat["properties"]["tmax"],
                "tmin": temp_feat["properties"]["tmin"],
            }
        )

# Convert the list of a Pandas Dataframe
temp_df = pandas.DataFrame(temp_data)
# Convert the date column to a datatime object
temp_df["date"] = pandas.to_datetime(temp_df["date"])
# Sort the Dataframe by the date
temp_df = temp_df.sort_values("date")
temp_df

In [ ]:
# Save the pandas dataframes to the google drive
temp_df.to_excel(
    "/content/drive/MyDrive/burnt_veg/burnt_area_noaa_temp_data.xlsx", index=False
)
